## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [6]:
class myModel:
    def __init__(self, input_dim=784, hidden_dim=128, output_dim=10):
        # 初始化模型参数
        self.W1 = tf.Variable(tf.random.normal([input_dim, hidden_dim], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([hidden_dim]))
        self.W2 = tf.Variable(tf.random.normal([hidden_dim, output_dim], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([output_dim]))
    
    def __call__(self, x):
        # 展平输入数据
        x = tf.reshape(x, (x.shape[0], -1))  # [batch_size, 28*28]
        # 实现模型的前向传播
        h1 = tf.matmul(x, self.W1) + self.b1
        h1 = tf.nn.relu(h1)  # ReLU 激活函数
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [7]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [10]:
train_data, test_data = mnist_dataset()
for epoch in range(200):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 1.3708534 ; accuracy 0.6737
epoch 1 : loss 1.3664372 ; accuracy 0.67503333
epoch 2 : loss 1.3620449 ; accuracy 0.6764
epoch 3 : loss 1.357677 ; accuracy 0.67793334
epoch 4 : loss 1.3533326 ; accuracy 0.6791
epoch 5 : loss 1.3490132 ; accuracy 0.6802
epoch 6 : loss 1.3447171 ; accuracy 0.6812
epoch 7 : loss 1.3404461 ; accuracy 0.6823
epoch 8 : loss 1.3361984 ; accuracy 0.6836
epoch 9 : loss 1.3319744 ; accuracy 0.68486667
epoch 10 : loss 1.3277737 ; accuracy 0.6861
epoch 11 : loss 1.3235961 ; accuracy 0.6875833
epoch 12 : loss 1.3194422 ; accuracy 0.68881667
epoch 13 : loss 1.3153117 ; accuracy 0.68995
epoch 14 : loss 1.3112041 ; accuracy 0.69116664
epoch 15 : loss 1.3071193 ; accuracy 0.6921667
epoch 16 : loss 1.3030576 ; accuracy 0.6935833
epoch 17 : loss 1.299019 ; accuracy 0.69451666
epoch 18 : loss 1.2950032 ; accuracy 0.69551665
epoch 19 : loss 1.2910104 ; accuracy 0.6965333
epoch 20 : loss 1.28704 ; accuracy 0.6977
epoch 21 : loss 1.2830914 ; accuracy 0.69838333
e